In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# Evaluation of Formulas from First Order Logic

In this notebook we show how formulas from first order logic can be evaluated in Python.

## The Axioms of Group Theory

To have a nontrivial example of formulas, we use the formulas from 
[group theory](https://en.wikipedia.org/wiki/Group_theory).  
A *group* is defined as a triple 
$$ \langle G, \mathrm{e}, * \rangle $$
where 
- $G$ is a non-empty set,
- $\mathrm{e}$ is an element from $G$, and
- $*:G \times G \rightarrow G$ is a binary function on $G$.
- Furthermore, the following axioms have to be satisfied:
  * $\forall x: \mathrm{e} * x = x$,
  * $\forall x: \exists{y}: y * x = \mathrm{e}$,
  * $\forall x: \forall y: \forall z: (x * y) * z = x * (y * z)$.
- A group is <em style="color:blue">commutative</em> if, additionally, the following formula is satisfied:
  $$\forall x: \forall y: x * y = y * x. $$

In [ ]:
import folParser as fp

Our parser distinguishes variables and function symbol as follows:
- A word starting with a lower case letter is interpreted as a *variable*.
- A word starting with an upper case letter is assumed to be a *function* or 
  *predicate symbol*.

Therefore, we represent the symbols from group theory as follows:
- The neutral element $\mathrm{e}$ of group theory is represented as the nullary function symbol `E`.
- As our parser does not support using the symbol $*$ as a binary operator, we will use the function symbol     
  `Multiply` to represent this operator.
- The predicate symbol $=$ is repesented as `Equals`

Then the formulas of group theory can be represented as follows:

In [ ]:
G1 = '∀x:Equals(Multiply(E(),x),x)'

In [ ]:
G2 = '∀x:∃y:Equals(Multiply(y,x),E())'

In [ ]:
G3 = '∀x:∀y:∀z:Equals(Multiply(Multiply(x,y),z), Multiply(x,Multiply(y,z)))'

In [ ]:
G4 = '∀x:∀y:Equals(Multiply(x,y), Multiply(y,x))'

The function $\texttt{parse}(s)$ takes a string $s$ and converts it into a nested tuple.

In [ ]:
def parse(s):
    "Parse string s as fol formula."
    p = fp.LogicParser(s, {'x', 'y', 'z'})
    return p.parse()

In [ ]:
F1 = parse(G1)
F1

In [ ]:
F2 = parse(G2)
F2

In [ ]:
F3 = parse(G3)
F3

In [ ]:
F4 = parse(G4)
F4

## A Structure for Group Theory

The smallest group has just two elements.  We define these elements to be the numbers `0` and  `1`.  We store these elements in the variables  <tt>a</tt> and <tt>b</tt>.  Then, we can define the universe <tt>U</tt> as follows:

In [ ]:
U = { 0, 1 }

Next, we need to define the nullary function that represents the nullary function <tt>E</tt>.  We define this function as a dictionary mapping the empty tuple into the element <tt>a</tt>. 

In [ ]:
NeutralElement = { (): 0 }

The binary function symbol <tt>Multiply</tt> is implemented as the dictionary <tt>Product</tt>:

In [ ]:
Product = { (0, 0): 0,  (0, 1): 1,  (1, 0): 1,  (1, 1): 0 }

The predicate symbol <tt>Equals</tt> is implemented as the binary relation <tt>Identity</tt>.

In [ ]:
Identity = { (x, x) for x in U }
Identity

Now the interpretation $\mathcal{J}$ can be implemented as a dictionary.

In [ ]:
J = { "E": NeutralElement, "Multiply": Product, "Equals": Identity }

Next, we define the *first order structure* $\mathcal{S}$.

In [ ]:
S = (U, J)
S

Finally, we define an *interpretation* $\mathcal{I}$ of the variables $x$, $y$, and $z$. 

In [ ]:
I = { "x": 0, "y": 1, "z": 0 } 
I

## Functions to Evaluate Formulas

The procedure $\texttt{evalTerm}(t, \mathcal{S}, \mathcal{I})$ evaluates the term $t$ in the structure $\mathcal{S}$ using the variable assignment $\mathcal{I}$.

In [ ]:
L = [1, 2, 3, 4]
x, *r = L
x, r

In [ ]:
def evalTerm(t, S, I):
    if isinstance(t, str):  # t is a variable
        return I[t]
    _, J     = S      # J is the dictionary of interpretations
    f, *args = t      # function symbol and arguments
    fJ       = J[f]   # interpretation of function symbol
    argVals  = evalTermTuple(args, S, I)
    return fJ[argVals]

The procedure $\texttt{evalTermTuple}(\texttt{Ts}, \mathcal{S}, \mathcal{I})$ evaluates the tuple of terms $\texttt{Ts}$, given the structure $\mathcal{S}$ and the variable assignment $\mathcal{I}$.

In [ ]:
def evalTermTuple(Ts, S, I):
    return tuple(evalTerm(t, S, I) for t in Ts)

In [ ]:
t = parse('Multiply(E(),x)')
t

In [ ]:
evalTerm(t, S, I)

This procedure evaluates the atomic formula a in the structure S using the variable assignment I.

In [ ]:
def evalAtomic(a, S, I):
    _, J     = S     # J is the dictionary of interpretations
    p, *args = a     # predicate symbol and arguments
    pJ       = J[p]  # interpretation of predicate symbol
    argVals  = evalTermTuple(args, S, I)
    return argVals in pJ

In [ ]:
f = parse('Equals(Multiply(E(),x),x)')
f

In [ ]:
evalAtomic(f, S, I)

Given a variable assignment $\mathcal{I}$, a variable $x$, and an element $c$ from the universe $\mathcal{U}$, the function $\texttt{modify}(\mathcal{I}, x, c)$ computes the variable assignment $\mathcal{I}[x/c]$ which is defined for all variables $y$ as follows:
$$ I[x/c](y) = \left\{ \begin{array}{ll}
                        c     & \mbox{if $x = y$,}  \\
                        I(y)  & \mbox{otherwise.}
                        \end{array}
               \right.
$$

In [ ]:
def modify(I, x, c):
    J = I.copy() # do not modify I
    J[x] = c
    return J

Given a first order logic formula $F$, a structure $\mathcal{S}$, and a variable assignment $\mathcal{I}$, the function $\texttt{evalFormula}(F, \mathcal{S}, \mathcal{I})$ computes the truth value of the formula $F$.

In [ ]:
def evalFormula(F, S, I):
    U, _ = S # U is the universe
    if F[0] == '⊤': return True
    if F[0] == '⊥': return False
    if F[0] == '¬': return not evalFormula(F[1], S, I)
    if F[0] == '∧': return evalFormula(F[1], S, I) and evalFormula(F[2], S, I)
    if F[0] == '∨': return evalFormula(F[1], S, I) or evalFormula(F[2], S, I)
    if F[0] == '→': return not evalFormula(F[1], S, I) or evalFormula(F[2], S, I)
    if F[0] == '↔': return evalFormula(F[1], S, I) == evalFormula(F[2], S, I)
    if F[0] == '∀': 
        x, G = F[1:] 
        return all({ evalFormula(G, S, modify(I, x, c)) for c in U })
    if F[0] == '∃':
        x, G = F[1:] 
        return any({ evalFormula(G, S, modify(I, x, c)) for c in U })
    return evalAtomic(F, S, I) 

## Checking whether $\mathcal{S}$ is a Group

In [ ]:
print(f"evalFormula({G1}, S, I) = {evalFormula(F1, S, I)}")
print(f"evalFormula({G2}, S, I) = {evalFormula(F2, S, I)}")
print(f"evalFormula({G3}, S, I) = {evalFormula(F3, S, I)}")
print(f"evalFormula({G4}, S, I) = {evalFormula(F4, S, I)}")

This shows that the structure $\mathcal{S}$ defined above is indeed a group.  Furthermore, it is a *commutative* group.

## Another Example

Let's show that the formula $\forall x: \exists y:p(x,y) \rightarrow \exists y:\forall x:p(x,y)$ is not universally valid, i.e. let's show the following:
$$ \not\models \forall x: \exists y:p(x,y) \rightarrow \exists y:\forall x:p(x,y) $$

In [ ]:
G = '∀x:∃y:P(x,y)→∃y:∀x:P(x,y)'

In [ ]:
F = parse(G)
F

Our aim is to construct a structure $\mathcal{S} = \langle\mathcal{U}, \mathcal{J} \rangle$  such that 
$$\mathcal{S}(F) = \mathtt{False}.$$ 

In [ ]:
U = {0, 1}

In [ ]:
pJ = { (0, 0), (1, 1) }

In [ ]:
J = { 'P': pJ }

In [ ]:
S = (U, J)

In [ ]:
I = { 'x': 0, 'y': 0 }

In [ ]:
evalFormula(F, S, I)